In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [15]:
## num_lines = sum(1 for line in tqdm(open('darach_data.lin')))
# print(num_lines)
# num_lines = 623161873

In [3]:
#print((vdat[vdat[0]==' BIBTEX ERROR: Line list contains a key not found in VALD3 BibTeX file: IST10  ']))

In [4]:
vdat = pd.read_csv('darach_data.lin', skiprows=594129240, nrows=29032460, header=None, skip_blank_lines=False, dtype=str)
print('1/6 - Data imported')

1/6 - Data imported


In [6]:
vdat = vdat.drop(13,axis=1)
vdatc = vdat.copy()
vdat = vdat[vdat[12].notna()]
#vdat = vdat[0:100000]
vdat = vdat.reset_index(drop=True)
#pd.options.display.max_rows = len(vdat[0])
#vdat
print('2/6 - Data sorted')

2/6 - Data sorted


In [7]:
el_list = pd.read_csv('elements.txt',delimiter=' ',header=None)
numbers = range(len(el_list[0])+1)
el_dict = dict(zip(el_list[0], numbers[1:]))

el = vdat[0].str.split(" ", n = 1, expand = True)
el[0] = el[0].replace({'\'':''}, regex=True)

for k,v in el_dict.items():
    el[0][el[0] == k] = v

# for i in tqdm(range(len(el[0]))):
#     e = el[0][i]
#     if e in el_dict:
#         el[0][i] = el_dict[e]
#     else:
#         el[0][i] = 0

el[1] = (el[1].replace({'\'':''}, regex=True).astype(int) - 1).apply(lambda x: '{0:0>2}'.format(x))
elm = el[0].map(str) + '.' + el[1].map(str)

print('3/6 - Elements done')

3/6 - Elements done


In [8]:
labels = vdatc.copy()
labels = labels[~labels[12].notna()]#[0:300000]
labels = pd.Series(labels.fillna('').values.tolist()).str.join(',')
labels = labels.replace({',,+':''}, regex=True)
labels = labels.replace({'\'  ':''}, regex=True)
labels = labels.replace({'\'':''}, regex=True)
labels = labels.replace({'  +':' '}, regex=True)

label_l = labels.iloc[::3]
label_l = label_l.reset_index(drop=True)
label_u = labels.iloc[1::3]
label_u = label_u.reset_index(drop=True)

print('4/6 - Labels done')

4/6 - Labels done


In [9]:
wave = vdat[1].astype(float)
log_gf = (vdat[2].astype(float))
E_l = vdat[3].astype(float)
j_l = vdat[4].astype(float).apply(lambda x: '{0:0>2}'.format(x))
#label_l = pd.DataFrame(['2D' for s in range(len(vdat))])
E_u = vdat[5].astype(float)
j_u = vdat[6].astype(float).apply(lambda x: '{0:0>2}'.format(x))
#label_u = pd.DataFrame(['2D' for s in range(len(vdat))])
lande_l = vdat[7].astype(float)*1000
lande_l = lande_l.replace(99000.0,0)
lande_u = vdat[8].astype(float)*1000
lande_u = lande_u.replace(99000.0,0)
rad = vdat[10].astype(float)
stark = vdat[11].astype(float)
waals = vdat[12].astype(float)
zeroes = pd.DataFrame(np.zeros((len(vdat), 1)))

df_vald = pd.concat([wave, log_gf, elm, E_l, j_l, label_l, E_u, j_u, label_u, rad, stark, waals, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, lande_l, lande_u], axis=1)
df_vald.columns = range(df_vald.shape[1])
#df_vald
print('5/6 - Dataframe created')

5/6 - Dataframe created


In [10]:
df_vald.iloc[-3:]

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
7258112,99999.0,-5.284,26.00,7.6070,5.0,JK s6h 2+[5+]*,7.6194,6.0,JK s7i 4+[6+],6.37,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1060.0,1120.0
7258113,99999.0,-5.216,26.00,7.6070,6.0,JK s6h 2+[5+]*,7.6194,7.0,r(5D)7I739,6.37,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1210.0,1240.0
7258114,100000.0,-7.082,26.01,13.5204,6.5,LS 3d6.(3G).4d 2I,13.5328,6.5,LS 3d5.(2I).4s.4p.(3P*) 2I*,9.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1080.0,1100.0


In [11]:
df_vald = df_vald[~(df_vald[2].astype(float)<1)]
df_vald.to_csv('VALD_linelist32.dat', header=None, index=None)

In [12]:
# with open('VALD_linelist1.dat', 'w') as f: 
#     df_vald.to_string(f, col_space=10, index=False, header=None)

print('6/6 - Line list ready')

6/6 - Line list ready
